##Install All packages


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q --upgrade langchain pypdf chromadb google-generativeai langchain-google-genai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Import All required Packages

In [ ]:
import google.generativeai as genai
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Set up API key

In [ ]:
# Used to securely store your API key
from google.colab import userdata
#dotenv package to load the API key
from dotenv import load_dotenv

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY') #Make secrets keys with same name as "GOOGLE_API_KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
#Create a new .env file in the workspace and store the API key in it
!echo -e 'GOOGLE_API_KEY=Paste_keys_here' > .env

In [ ]:
load_dotenv()

True

#Load PDF files from Folder

In [ ]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/Colab Notebooks/pdfs")
data = loader.load()

In [ ]:
data[0]

Document(page_content="GENERATIVE AI WITH LLMs:  \n \nINTRODUCTION OF GENERATIVE AI:  \nA novel area of artificial intelligence called generative AI aims to develop models that produce \nfresh content in a manner similar to that of human creators. Large Language Models (LLMs), \nsuch as the GPT series, which are well-known for their capacity to comprehend and generate \nnatural language text, are essential to this subject.  \nEquipped with advanced neural network architectures, these models identify patterns and \nconnections across large datasets to produce logical and contextually appropriate language. \nCapable of handling anything from creative text to translation, LLMs are used in a variety of \nsectors such as text translation, text generation, text classification , text summarization , text to \nimage generation and also u se to improve chatbot interactions and support content \ndevelopment and analysis.  \nPeople are working on making these models better and fairer, so they can

#Split data into Text chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
content = "\n\n".join(str(page.page_content) for page in data)

In [ ]:
texts = text_splitter.split_text(content)

In [ ]:
print(len(texts))

96


In [ ]:
texts[0]

'GENERATIVE AI WITH LLMs:  \n \nINTRODUCTION OF GENERATIVE AI:  \nA novel area of artificial intelligence called generative AI aims to develop models that produce \nfresh content in a manner similar to that of human creators. Large Language Models (LLMs), \nsuch as the GPT series, which are well-known for their capacity to comprehend and generate \nnatural language text, are essential to this subject.  \nEquipped with advanced neural network architectures, these models identify patterns and \nconnections across large datasets to produce logical and contextually appropriate language. \nCapable of handling anything from creative text to translation, LLMs are used in a variety of \nsectors such as text translation, text generation, text classification , text summarization , text to \nimage generation and also u se to improve chatbot interactions and support content \ndevelopment and analysis.  \nPeople are working on making these models better and fairer, so they can do even more things'

#Download Embeddings model

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

#Create embeddings for each of the Text chunks and Store them in Vectorstore-chromadb

In [ ]:
vector_store = Chroma.from_texts(texts, embeddings).as_retriever()

#Create a Prompt Template

In [ ]:
prompt_template = """
  Please answer the question in as much detail as possible based on the provided context.
  Ensure to include all relevant details. If the answer is not available in the provided context,
  kindly respond with "The answer is not available in the context." Please avoid providing incorrect answers.
\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

#Load the model

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.1)


In [ ]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [ ]:
question = "what is generative AI?"
docs = vector_store.get_relevant_documents(question)

response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
# response

In [ ]:
to_markdown(response["output_text"])

> Generative AI is a novel area of artificial intelligence that aims to develop models capable of producing fresh content in a manner similar to human creators. These models are equipped with advanced neural network architectures that enable them to identify patterns and connections across large datasets, allowing them to generate logical and contextually appropriate language. Generative AI systems are used in various sectors, including text translation, text generation, text classification, text summarization, text-to-image generation, and improving chatbot interactions.

In [ ]:
question = "Number of foreign and domestic tourists arrivals in Gilgit-Baltistan from 2007 to 2015?"
docs = vector_store.get_relevant_documents(question)

response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
# response

In [ ]:
to_markdown(response["output_text"])

> The number of foreign and domestic tourists arrivals in Gilgit-Baltistan from 2007 to 2015 is as follows:
> 
> | Year | Total No. of Tourists | Foreign Tourists | Domestic Tourists |
> |---|---|---|---|
> | 2007 | 34,108 | 10,338 (30.3%) | 23,770 (69.69%) |
> | 2008 | 62,544 | 8,504 (13.60%) | 54,040 (86.40%) |
> | 2009 | 62,341 | 7,739 (12.42%) | 54,602 (87.58%) |
> | 2010 | 53,028 | 7,728 (14.58%) | 45,300 (85.42%) |
> | 2011 | 66,475 | 5,242 (7.89%) | 61,233 (92.11%) |
> | 2012 | 33,217 | 4,324 (13.00%) | 28,893 (87.00%) |
> | 2013 | 56,415 | 4,501 (8.00%) | 51,914 (92.00%) |
> | 2014 | 53,746 | 3,442 (6.40%) | 50,304 (93.60%) |
> | 2015 | 204,733 | 4,082 (2.0%) | 200,651 (98.00%) |

In [ ]:
question = "what is Physicochemical test result in table 6?"
docs = vector_store.get_relevant_documents(question)

response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
# response

In [ ]:
to_markdown(response["output_text"])

> The physicochemical test results in Table 6 include pH, turbidity, electric conductivity, DO, and hardness. The pH values range from 6.5 to 8.5, turbidity values range from 1.28 to 66.36 NTU, electric conductivity values range from 3.12 to 138.5 μS/cm, DO values range from 5.66 to 5.99, and hardness values range from 55.36 to 666.8 mg/L.

In [ ]:
question = "what is Economic impacts of tourism?"
docs = vector_store.get_relevant_documents(question)

response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
# response

In [ ]:
to_markdown(response["output_text"])

> On the positive side, tourism is an important source of welfare in terms of being a good source of income and employment for local people. Tourism can play a role in the economic growth of a country and is also the source of foreign exchange income for developing countries, contributing to the alleviation of poverty with major fiscal activities.

In [ ]:
question = "explain PEFT?"
docs = vector_store.get_relevant_documents(question)

response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
# response

In [ ]:
to_markdown(response["output_text"])

> PEFT (Parameter-Efficient Fine-Tuning) is a framework that integrates various techniques to enable efficient fine-tuning of large language models (LLMs) with limited resources. It focuses on adapter-based methods, which involve adding small, trainable adapters to pre-trained LLMs to reduce the number of parameters that need to be updated during fine-tuning.
> 
> One specific adapter-based method within PEFT is LoRA (Low-Rank Adaptation), which uses low-rank matrices for efficient adaptation. PEFT also includes techniques such as 4-bit quantization, which compresses LLM weights and activations to 4 bits instead of 32 or 16 bits, dramatically reducing memory footprint and speeding up computations.
> 
> Additionally, PEFT employs high-precision compute to preserve accuracy by performing computations using higher precision (e.g., 16-bit) to avoid quantization errors. Overall, PEFT aims to enable efficient fine-tuning of LLMs with limited resources by integrating various techniques that reduce memory requirements and preserve performance.